# Himalayan Expeditions Data Visualization

**Autor: Claude 3.7 Sonnet**

Este notebook contiene el análisis y visualización de datos de expediciones al Himalaya, según los requisitos del PRD.

## Objetivos

Desarrollar un sistema de visualización interactivo multi-vista que permita responder a las siguientes preguntas:

1. ¿Qué rutas tienen las tasas de éxito más altas y más bajas para diferentes picos?
2. ¿Qué países han liderado más expediciones en diferentes períodos de tiempo?
3. ¿Las expediciones más largas tienen una mayor probabilidad de alcanzar la cumbre con éxito para cada pico? ¿Esto cambia según la temporada?
4. ¿Cómo han evolucionado las razones de terminación a lo largo de los años para cada pico?

## Configuración inicial

In [46]:
# Importar bibliotecas necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import altair as alt

# Aumentar el límite de filas para Altair
alt.data_transformers.disable_max_rows()

# Configurar opciones de visualización para pandas
pd.set_option('display.max_columns', None)

In [47]:
# Acceso a Google Drive
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/Colab Notebooks/VD-Lab2/'
input_data = path+'input_data/'
output_data = path+'output_data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Cargar y examinar los datos

In [48]:
# Cargar los conjuntos de datos
exped_df = pd.read_csv(input_data+'exped_tidy.csv', encoding='latin-1')
peaks_df = pd.read_csv(input_data+'peaks_tidy.csv', encoding='latin-1')
coords_df = pd.read_csv(input_data+'unique_peaks_coords.csv', encoding='latin-1')

# Mostrar información básica sobre los datasets
print("Dataset de expediciones:")
print(f"Número de registros: {exped_df.shape[0]}")
print(f"Número de columnas: {exped_df.shape[1]}")

print("\nDataset de picos:")
print(f"Número de registros: {peaks_df.shape[0]}")
print(f"Número de columnas: {peaks_df.shape[1]}")

print("\nDataset de coordenadas:")
print(f"Número de registros: {coords_df.shape[0]}")
print(f"Número de columnas: {coords_df.shape[1]}")

Dataset de expediciones:
Número de registros: 882
Número de columnas: 69

Dataset de picos:
Número de registros: 480
Número de columnas: 29

Dataset de coordenadas:
Número de registros: 116
Número de columnas: 4


In [49]:
# Examinar las primeras filas de cada conjunto de datos
print("Primeras filas del dataset de expediciones:")
exped_df.head()

Primeras filas del dataset de expediciones:


,EXPID,PEAKID,YEAR,SEASON,SEASON_FACTOR,HOST,HOST_FACTOR,ROUTE1,ROUTE2,ROUTE3,ROUTE4,NATION,LEADERS,SPONSOR,SUCCESS1,SUCCESS2,SUCCESS3,SUCCESS4,ASCENT1,ASCENT2,ASCENT3,ASCENT4,CLAIMED,DISPUTED,COUNTRIES,APPROACH,BCDATE,SMTDATE,SMTTIME,SMTDAYS,TOTDAYS,TERMDATE,TERMREASON,TERMREASON_FACTOR,TERMNOTE,HIGHPOINT,TRAVERSE,SKI,PARAPENTE,CAMPS,ROPE,TOTMEMBERS,SMTMEMBERS,MDEATHS,TOTHIRED,SMTHIRED,HDEATHS,NOHIRED,O2USED,O2NONE,O2CLIMB,O2DESCENT,O2SLEEP,O2MEDICAL,O2TAKEN,O2UNKWN,OTHERSMTS,CAMPSITES,ROUTEMEMO,ACCIDENTS,ACHIEVMENT,AGENCY,COMRTE,STDRTE,PRIMRTE,PRIMMEM,PRIMREF,PRIMID,CHKSUM
0,EVER20101,EVER,2020,1,Spring,2,China,N Col-NE Ridge,NaN,NaN,NaN,China,Tibetan Rope-Fixing,Tibetan Rope-Fixing Everest North 2020,True,False,False,False,NaN,NaN,NaN,NaN,False,False,NaN,Lhasa->Tingri->Everest BC,NaN,2020-05-26,1515.0,0,0,NaN,1,Success (main peak),NaN,8849,False,False,False,3,0,0,0,0,6,6,0,False,True,False,True,False,True,False,False,False,NaN,"BC,ABC,C1,C2,C3,Smt(26/05)",NaN,NaN,NaN,Holy Mountain Adventure,True,True,False,False,False,NaN,2465291
1,EVER20102,EVER,2020,1,Spring,2,China,N Col-NE Ridge,NaN,NaN,NaN,China,Ci Luo (Tselo),Chinese Mount Everest Survey Team,True,False,False,False,NaN,NaN,NaN,NaN,False,False,NaN,NaN,NaN,2020-05-27,945.0,0,0,NaN,1,Success (main peak),NaN,8849,False,False,False,3,0,12,8,0,0,0,0,False,True,False,True,False,True,False,False,False,NaN,"BC,ABC,C1,C2,C3,Smt(27/05)",221011.0,NaN,NaN,NaN,True,True,False,False,False,NaN,2465292
2,EVER20103,EVER,2020,1,Spring,2,China,N Col-NE Ridge,NaN,NaN,NaN,China,Tsering Samdrup,Holy Mountain Adventure Everest Expedition 2020,True,False,False,False,NaN,NaN,NaN,NaN,False,False,NaN,Lhasa->Tingri->Everest BC,2020-04-23,2020-05-28,545.0,35,38,2020-05-31,1,Success (main peak),NaN,8849,False,False,False,3,0,20,14,0,22,21,0,False,True,False,True,False,True,False,False,False,NaN,"BC(23/04,5200m),IC(26/04,5800m),ABC(05/01,6500...",203869.0,NaN,NaN,Holy Mountain Adventure,True,True,False,False,False,NaN,2465293
3,AMAD20301,AMAD,2020,3,Autumn,1,Nepal,SW Ridge,NaN,NaN,NaN,Nepal,Chhang Dawa Sherpa,Seven Summit Treks Ama Dablam Expedition 2020,True,False,False,False,NaN,NaN,NaN,NaN,False,False,"Canada, Czech Republic, France, Poland, Russia...",NaN,2020-11-09,2020-11-10,1300.0,1,0,NaN,1,Success (main peak),NaN,6814,False,False,False,2,0,14,9,0,19,14,0,False,False,True,False,False,False,False,False,False,NaN,"BC(09/11,4450m),C1(5600m),C2(5900m),Smt(10,12-...",NaN,NaN,NaN,Seven Summit Treks,True,False,False,False,False,NaN,2463299
4,AMAD20302,AMAD,2020,3,Autumn,1,Nepal,SW Ridge,NaN,NaN,NaN,USA,Garrett Madison,Madison Mountaineering Ama Dablam Expedition 2020,True,False,False,False,NaN,NaN,NaN,NaN,False,False,"Canada, Qatar",Lukla->Pangboche->Ama Dablam BC,2020-11-01,2020-11-10,1300.0,9,11,2020-11-12,1,Success (main peak),NaN,6814,False,False,False,2,0,6,6,0,8,8,0,False,False,True,False,False,False,False,False,False,NaN,"BC(01/11,4700m),C1(04/11,5600m),C2(10/11,6000m...",29755.0,NaN,NaN,Himalayan Guides,True,False,False,False,False,NaN,2463299


In [50]:
print("Primeras filas del dataset de picos:")
peaks_df.head()

Primeras filas del dataset de picos:


,PEAKID,PKNAME,PKNAME2,LOCATION,HEIGHTM,HEIGHTF,HIMAL,HIMAL_FACTOR,REGION,REGION_FACTOR,OPEN,UNLISTED,TREKKING,TREKYEAR,RESTRICT,PHOST,PHOST_FACTOR,PSTATUS,PSTATUS_FACTOR,PEAKMEMO,PYEAR,PSEASON,PEXPID,PSMTDATE,PCOUNTRY,PSUMMITERS,PSMTNOTE,REFERMEMO,PHOTOMEMO
0,AMAD,Ama Dablam,Amai Dablang,Khumbu Himal,6814,22356,12,Khumbu,2,Khumbu-Rolwaling-Makalu,True,False,False,NaN,NaN,1,Nepal only,2,Climbed,8.0,1961.0,1,AMAD61101,Mar 13,"New Zealand, USA, UK","Mike Gill, Wally Romanes, Barry Bishop, Michae...",NaN,NaN,13.0
1,AMPG,Amphu Gyabjen,Amphu Gyabien,Khumbu Himal (N of Ama Dablam),5630,18471,12,Khumbu,2,Khumbu-Rolwaling-Makalu,True,False,False,NaN,Opened in 2002,1,Nepal only,2,Climbed,20.0,1953.0,1,AMPG53101,Apr 11,UK,"John Hunt, Tom Bourdillon",NaN,NaN,NaN
2,ANN1,Annapurna I,NaN,Annapurna Himal,8091,26545,1,Annapurna,5,Annapurna-Damodar-Peri,True,False,False,NaN,NaN,1,Nepal only,2,Climbed,23.0,1950.0,1,ANN150101,Jun 03,France,"Maurice Herzog, Louis Lachenal",NaN,25.0,26.0
3,ANN2,Annapurna II,NaN,Annapurna Himal,7937,26040,1,Annapurna,5,Annapurna-Damodar-Peri,True,False,False,NaN,NaN,1,Nepal only,2,Climbed,31.0,1960.0,1,ANN260101,May 17,"UK, Nepal","Richard Grant, Chris Bonington, Ang Nyima Sherpa",NaN,33.0,34.0
4,ANN3,Annapurna III,NaN,Annapurna Himal,7555,24787,1,Annapurna,5,Annapurna-Damodar-Peri,True,False,False,NaN,NaN,1,Nepal only,2,Climbed,35.0,1961.0,1,ANN361101,May 06,India,"Mohan S. Kohli, Sonam Gyatso, Sonam Girmi",NaN,NaN,37.0


In [51]:
print("Primeras filas del dataset de coordenadas:")
coords_df.head()

Primeras filas del dataset de coordenadas:


,PEAKID,PKNAME,LATITUDE,LONGITUDE
0,EVER,Everest,27.9881,86.9250
1,AMAD,Ama Dablam,27.8610,86.8610
2,BARU,Baruntse,27.8711,86.9825
3,CHOY,Cho Oyu,28.0944,86.6608
4,GYLZ,Gyalzen Peak,28.3583,85.4083


## Limpieza y preprocesamiento de datos

In [52]:
# Verificar valores nulos en las columnas clave
print("Valores nulos en el dataset de expediciones (columnas clave):")
key_columns = ['PEAKID', 'YEAR', 'SEASON_FACTOR', 'ROUTE1', 'ROUTE2', 'ROUTE3', 'ROUTE4',
               'SUCCESS1', 'SUCCESS2', 'SUCCESS3', 'SUCCESS4', 'TERMREASON_FACTOR', 'TOTDAYS', 'HOST_FACTOR']
exped_df[key_columns].isnull().sum()

Valores nulos en el dataset de expediciones (columnas clave):


,0
PEAKID,0
YEAR,0
SEASON_FACTOR,0
ROUTE1,20
ROUTE2,839
ROUTE3,882
ROUTE4,882
SUCCESS1,0
SUCCESS2,0
SUCCESS3,0


In [53]:
# Limpieza del dataset de expediciones
df_clean = exped_df.copy()

# Unificar valores nulos para rutas (NaN, NA, cadenas vacías)
route_columns = ['ROUTE1', 'ROUTE2', 'ROUTE3', 'ROUTE4']
for col in route_columns:
    df_clean[col] = df_clean[col].replace('NA', np.nan).replace('', np.nan)

# Convertir columnas de éxito en booleanas explícitas
success_columns = ['SUCCESS1', 'SUCCESS2', 'SUCCESS3', 'SUCCESS4']
for col in success_columns:
    df_clean[col] = df_clean[col].fillna(False).astype(bool)

# Asegurarse de que TOTDAYS sea numérico y manejar valores faltantes
df_clean['TOTDAYS'] = pd.to_numeric(df_clean['TOTDAYS'], errors='coerce')

# Crear año como entero para facilitar filtrado
df_clean['YEAR_INT'] = pd.to_numeric(df_clean['YEAR'], errors='coerce')

# Verificar que al menos tengamos PEAKID y YEAR en todos los registros
df_clean = df_clean.dropna(subset=['PEAKID', 'YEAR_INT'])

# Crear un indicador de éxito general para la expedición
df_clean['ANY_SUCCESS'] = df_clean[success_columns].any(axis=1)

print(f"Registros originales: {exped_df.shape[0]}")
print(f"Registros después de limpieza: {df_clean.shape[0]}")

Registros originales: 882
Registros después de limpieza: 882


In [55]:
# Merge con datos de picos y coordenadas para tener información completa
df_merged = pd.merge(df_clean, peaks_df[['PEAKID', 'PKNAME', 'HEIGHTM', 'HIMAL_FACTOR', 'REGION_FACTOR']],
                     on='PEAKID', how='left')
df_merged = pd.merge(df_merged, coords_df, on='PEAKID', how='left').rename(columns={'PKNAME_x': 'PKNAME'})

# Crear una versión filtrada con solo los picos más populares para algunas visualizaciones
peak_counts = df_merged['PEAKID'].value_counts()
top_peaks = peak_counts[peak_counts >= 30].index.tolist()
df_top_peaks = df_merged[df_merged['PEAKID'].isin(top_peaks)]

print(f"Número de picos únicos en el dataset: {df_merged['PEAKID'].nunique()}")
print(f"Número de picos populares (≥30 expediciones): {len(top_peaks)}")
print("\nPicos populares:")
peak_names = df_merged[df_merged['PEAKID'].isin(top_peaks)][['PEAKID', 'PKNAME']].drop_duplicates()
peak_names

Número de picos únicos en el dataset: 116
Número de picos populares (≥30 expediciones): 6

Picos populares:


,PEAKID,PKNAME
0,EVER,Everest
3,AMAD,Ama Dablam
14,HIML,Himlung Himal
18,MANA,Manaslu
36,DHA1,Dhaulagiri I
96,LHOT,Lhotse


## Preparación de datos específicos para las visualizaciones

### 1. Tasas de éxito por rutas y picos

In [56]:
# Expandir datos para obtener pares ruta-éxito para cada expedición
route_success_pairs = []

for _, row in df_merged.iterrows():
    for i in range(1, 5):  # Para ROUTE1-4 y SUCCESS1-4
        route_col = f'ROUTE{i}'
        success_col = f'SUCCESS{i}'

        if pd.notna(row[route_col]):
            route_success_pairs.append({
                'EXPID': row['EXPID'],
                'PEAKID': row['PEAKID'],
                'PKNAME': row['PKNAME'],
                'YEAR': row['YEAR'],
                'SEASON_FACTOR': row['SEASON_FACTOR'],
                'ROUTE': row[route_col],
                'SUCCESS': row[success_col],
                'HEIGHTM': row['HEIGHTM']
            })

routes_df = pd.DataFrame(route_success_pairs)

# Calcular tasas de éxito por ruta y pico
route_success_rates = routes_df.groupby(['PEAKID', 'PKNAME', 'ROUTE']).agg(
    total_attempts=('SUCCESS', 'count'),
    successful_attempts=('SUCCESS', 'sum'),
    height=('HEIGHTM', 'first')
).reset_index()

route_success_rates['success_rate'] = route_success_rates['successful_attempts'] / route_success_rates['total_attempts']

# Filtrar solo rutas con al menos 5 intentos para tener estadísticas más significativas
route_success_rates = route_success_rates[route_success_rates['total_attempts'] >= 5]

# Ordenar por tasa de éxito
route_success_rates = route_success_rates.sort_values('success_rate', ascending=False)

print(f"Número total de combinaciones ruta-pico: {route_success_rates.shape[0]}")
route_success_rates.head()

Número total de combinaciones ruta-pico: 16


,PEAKID,PKNAME,ROUTE,total_attempts,successful_attempts,height,success_rate
33,CHOY,Cho Oyu,NW Ridge,6,6,8188,1.000000
104,LHOT,Lhotse,S Col-W Face,48,47,8516,0.979167
1,AMAD,Ama Dablam,SW Ridge,145,136,6814,0.937931
56,EVER,Everest,S Col-SE Ridge,171,156,8849,0.912281
6,ANN1,Annapurna I,N Face,20,18,8091,0.900000


### 2. Expediciones por país y período

In [57]:
# Crear bins para décadas
df_merged['decade'] = (df_merged['YEAR_INT'] // 10) * 10
df_merged['decade'] = df_merged['decade'].astype(str) + 's'

# Contar expediciones por país y década
country_expeditions = df_merged.groupby(['HOST_FACTOR', 'decade']).size().reset_index(name='count')

# Identificar los países más activos
top_countries = df_merged['HOST_FACTOR'].value_counts().head(10).index.tolist()

# Filtrar para los países más activos
country_expeditions_top = country_expeditions[country_expeditions['HOST_FACTOR'].isin(top_countries)]

print("Países con más expediciones:")
print(df_merged['HOST_FACTOR'].value_counts().head(10))
country_expeditions_top.head()

Países con más expediciones:
HOST_FACTOR
Nepal    855
China     25
India      2
Name: count, dtype: int64


,HOST_FACTOR,decade,count
0,China,2020s,25
1,India,2020s,2
2,Nepal,2020s,855


### 3. Duración de expediciones y tasa de éxito

In [58]:
# Filtrar registros con información de duración válida
duration_df = df_merged.dropna(subset=['TOTDAYS', 'ANY_SUCCESS'])
duration_df = duration_df[duration_df['TOTDAYS'] > 0]

# Crear bins para duración
duration_bins = [0, 15, 30, 45, 60, 75, 90, 365]
duration_labels = ['1-15', '16-30', '31-45', '46-60', '61-75', '76-90', '90+']
duration_df['duration_bin'] = pd.cut(duration_df['TOTDAYS'], bins=duration_bins, labels=duration_labels, right=False)

# Calcular tasas de éxito por pico, temporada y bin de duración
duration_success = duration_df.groupby(['PEAKID', 'PKNAME', 'SEASON_FACTOR', 'duration_bin']).agg(
    total=('EXPID', 'count'),
    success=('ANY_SUCCESS', 'sum')
).reset_index()

duration_success['success_rate'] = duration_success['success'] / duration_success['total']

# Filtrar para tener al menos 3 expediciones en cada bin
duration_success = duration_success[duration_success['total'] >= 3]

print(f"Registros con información de duración: {duration_df.shape[0]}")
duration_success.head()

Registros con información de duración: 333


<ipython-input-58-0a6bada9970e>:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  duration_success = duration_df.groupby(['PEAKID', 'PKNAME', 'SEASON_FACTOR', 'duration_bin']).agg(


,PEAKID,PKNAME,SEASON_FACTOR,duration_bin,total,success,success_rate
0,AMAD,Ama Dablam,Autumn,1-15,14,13,0.928571
1,AMAD,Ama Dablam,Autumn,16-30,8,8,1.000000
8,AMAD,Ama Dablam,Spring,16-30,3,2,0.666667
2304,ANN1,Annapurna I,Spring,16-30,9,7,0.777778
9191,BARU,Baruntse,Spring,1-15,3,1,0.333333


### 4. Evolución de razones de terminación

In [59]:
# Filtrar registros con razones de terminación conocidas
termination_df = df_merged.dropna(subset=['TERMREASON_FACTOR'])

# Crear períodos de tiempo (por cada 5 años)
termination_df['period'] = (termination_df['YEAR_INT'] // 5) * 5
termination_df['period'] = termination_df['period'].astype(str) + '-' + (termination_df['period'] + 4).astype(str)

# Agrupar las razones menos comunes como "Other"
reason_counts = termination_df['TERMREASON_FACTOR'].value_counts()
common_reasons = reason_counts[reason_counts >= 100].index.tolist()
termination_df['reason_grouped'] = termination_df['TERMREASON_FACTOR'].apply(
    lambda x: x if x in common_reasons else 'Other reasons')

# Calcular distribución de razones por pico y período
term_evolution = termination_df.groupby(['PEAKID', 'PKNAME', 'period', 'reason_grouped']).size().reset_index(name='count')

# Calcular porcentajes dentro de cada pico y período
term_evolution = term_evolution.merge(
    term_evolution.groupby(['PEAKID', 'period'])['count'].sum().reset_index(name='total'),
    on=['PEAKID', 'period']
)
term_evolution['percentage'] = term_evolution['count'] / term_evolution['total'] * 100

print("Razones de terminación más comunes:")
print(reason_counts.head(10))
term_evolution.head()

Razones de terminación más comunes:
TERMREASON_FACTOR
Success (main peak)                                                             628
Bad weather (storms, high winds)                                                 78
Bad conditions (deep snow, avalanching, falling ice, or rock)                    54
Success (subpeak, foresummit)                                                    26
Illness, AMS, exhaustion, or frostbite                                           25
Unknown                                                                          23
Did not attempt climb                                                            12
Other                                                                            12
Route technically too difficult, lack of experience, strength, or motivation      9
Accident (death or serious injury)                                                5
Name: count, dtype: int64


,PEAKID,PKNAME,period,reason_grouped,count,total,percentage
0,AMAD,Ama Dablam,2020-2024,Other reasons,10,147,6.802721
1,AMAD,Ama Dablam,2020-2024,Success (main peak),137,147,93.197279
2,AMPG,Amphu Gyabjen,2020-2024,Success (main peak),1,1,100.000000
3,ANID,Anidesh Chuli,2020-2024,Other reasons,1,1,100.000000
4,ANN1,Annapurna I,2020-2024,Other reasons,3,22,13.636364


## Visualizaciones con Altair

### 1. Tasas de éxito por ruta y pico

In [60]:
# Filtrar para los 12 picos más populares para la visualización inicial
top_12_peaks = df_merged['PEAKID'].value_counts().head(12).index.tolist()
route_success_top = route_success_rates[route_success_rates['PEAKID'].isin(top_12_peaks)]

# Ordenar rutas por tasa de éxito dentro de cada pico
route_success_top = route_success_top.sort_values(['PEAKID', 'success_rate'], ascending=[True, False])

# Crear selector para picos
peak_selector = alt.selection_point(
    name='peak_selector',
    fields=['PEAKID'],
    # Use 'value' instead of 'init' for alt.selection_point in v5
    # Initialize value with a list containing the first peak ID
    value=[{'PEAKID': top_12_peaks[0]}],
    bind=alt.binding_select(options=top_12_peaks, name='Mountain Peak: ')
)

# Color para las barras de éxito
color_scale = alt.Scale(
    domain=[0, 0.25, 0.5, 0.75, 1.0],
    range=['#c22d2d', '#e77e16', '#ffb533', '#d9e03f', '#48c13d']
)

# Gráfico de barras para tasas de éxito por ruta
route_chart = alt.Chart(route_success_top).mark_bar().encode(
    x=alt.X('ROUTE:N', axis=alt.Axis(title='Route', labelAngle=-45), sort='-y'),
    y=alt.Y('success_rate:Q', axis=alt.Axis(title='Success Rate', format='.0%'), scale=alt.Scale(domain=[0, 1])),
    color=alt.Color('success_rate:Q', scale=color_scale, legend=alt.Legend(title='Success Rate')),
    tooltip=[
        alt.Tooltip('PKNAME:N', title='Peak'),
        alt.Tooltip('ROUTE:N', title='Route'),
        alt.Tooltip('success_rate:Q', title='Success Rate', format='.1%'),
        alt.Tooltip('successful_attempts:Q', title='Successful Attempts'),
        alt.Tooltip('total_attempts:Q', title='Total Attempts')
    ]
).properties(
    title=alt.TitleParams(text='Success Rates by Route', fontSize=16),
    width=600,
    height=400
).transform_filter(
    peak_selector
).add_selection(
    peak_selector
)

# Gráfico de información general sobre los picos
peak_info = alt.Chart(df_merged[df_merged['PEAKID'].isin(top_12_peaks)].drop_duplicates('PEAKID')).mark_bar().encode(
    x=alt.X('PEAKID:N', axis=alt.Axis(title='Peak ID')),
    y=alt.Y('HEIGHTM:Q', axis=alt.Axis(title='Height (m)')),
    color=alt.condition(peak_selector, alt.value('#1f77b4'), alt.value('lightgray')),
    tooltip=[
        alt.Tooltip('PKNAME:N', title='Peak Name'),
        alt.Tooltip('HEIGHTM:Q', title='Height (m)'),
        alt.Tooltip('REGION_FACTOR:N', title='Region'),
        alt.Tooltip('HIMAL_FACTOR:N', title='Mountain Range')
    ]
).properties(
    title=alt.TitleParams(text='Peak Heights', fontSize=16),
    width=600,
    height=200
).add_selection(
    peak_selector
)

# Combinar los gráficos
route_success_viz = alt.vconcat(peak_info, route_chart).resolve_scale(
    color='independent'
)

route_success_viz

<ipython-input-60-3fe439e75bb7>:42: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  ).add_selection(
<ipython-input-60-3fe439e75bb7>:61: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  ).add_selection(


alt.VConcatChart(...)

### 2. Expediciones lideradas por países a lo largo del tiempo

In [ ]:
# Selector para países
country_selector = alt.selection_multi(
    fields=['HOST_FACTOR'],
    bind='legend'
)

# Gráfico de líneas para la evolución de expediciones por país
countries_chart = alt.Chart(country_expeditions_top).mark_line(point=True).encode(
    x=alt.X('decade:N', axis=alt.Axis(title='Decade')),
    y=alt.Y('count:Q', axis=alt.Axis(title='Number of Expeditions')),
    color=alt.Color('HOST_FACTOR:N', legend=alt.Legend(title='Host Country')),
    strokeWidth=alt.condition(country_selector, alt.value(3), alt.value(1)),
    opacity=alt.condition(country_selector, alt.value(1), alt.value(0.2)),
    tooltip=[
        alt.Tooltip('HOST_FACTOR:N', title='Country'),
        alt.Tooltip('decade:N', title='Decade'),
        alt.Tooltip('count:Q', title='Expeditions')
    ]
).properties(
    title='Expeditions Led by Countries Over Time',
    width=800,
    height=500
).add_selection(
    country_selector
)

# Histograma apilado como complemento
countries_stacked = alt.Chart(country_expeditions_top).mark_bar().encode(
    x=alt.X('decade:N', axis=alt.Axis(title='Decade')),
    y=alt.Y('count:Q', axis=alt.Axis(title='Number of Expeditions')),
    color=alt.Color('HOST_FACTOR:N', legend=None),
    opacity=alt.condition(country_selector, alt.value(1), alt.value(0.2)),
    tooltip=[
        alt.Tooltip('HOST_FACTOR:N', title='Country'),
        alt.Tooltip('decade:N', title='Decade'),
        alt.Tooltip('count:Q', title='Expeditions')
    ]
).properties(
    title='Stacked View of Expeditions by Country',
    width=800,
    height=300
).add_selection(
    country_selector
)

# Combinar los gráficos
countries_viz = alt.vconcat(countries_chart, countries_stacked).resolve_scale(
    color='shared'
)

countries_viz

<ipython-input-30-3fbfc75538df>:2: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  country_selector = alt.selection_multi(
<ipython-input-30-3fbfc75538df>:23: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  ).add_selection(
<ipython-input-30-3fbfc75538df>:42: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  ).add_selection(


alt.VConcatChart(...)

### 3. Duración de expedición y tasa de éxito

In [61]:
# Crear selectores para pico y temporada
# Usar alt.selection_point y 'value' en lugar de 'init' para Altair v5
duration_peak_selector = alt.selection_point(
    name='peak_select',
    fields=['PEAKID'],
    # Changed 'init' to 'value' and passed a list containing a dictionary
    value=[{'PEAKID': top_peaks[0]}],
    bind=alt.binding_select(options=top_peaks, name='Peak: ')
)

# Obtener todas las temporadas disponibles
all_seasons = sorted(duration_df['SEASON_FACTOR'].dropna().unique().tolist())

# Use alt.selection_point and 'value' for season_selector as well
season_selector = alt.selection_point(  # Changed from selection_single
    name='season_select',
    fields=['SEASON_FACTOR'],
    # Changed 'init' to 'value'
    value=[{'SEASON_FACTOR': 'all'}],  # Value should be a list of dictionaries for point selection
    bind=alt.binding_select(options=['all'] + all_seasons, name='Season: ')
)

# Preparar datos para la visualización general
duration_avg = duration_df.groupby(['PEAKID', 'PKNAME', 'ANY_SUCCESS']).agg(
    avg_duration=('TOTDAYS', 'mean'),
    count=('EXPID', 'count')
).reset_index()

# Gráfico de dispersión para la relación general entre duración y éxito
scatter_chart = alt.Chart(duration_avg).mark_circle(size=100).encode(
    x=alt.X('avg_duration:Q', axis=alt.Axis(title='Average Expedition Duration (days)')),
    y=alt.Y('PKNAME:N', axis=alt.Axis(title='Peak')),
    color=alt.Color('ANY_SUCCESS:N',
                   scale=alt.Scale(domain=[True, False], range=['#48c13d', '#c22d2d']),
                   legend=alt.Legend(title='Summit Success')),
    size=alt.Size('count:Q', legend=alt.Legend(title='Number of Expeditions')),
    tooltip=[
        alt.Tooltip('PKNAME:N', title='Peak'),
        alt.Tooltip('ANY_SUCCESS:N', title='Success'),
        alt.Tooltip('avg_duration:Q', title='Avg. Duration (days)', format='.1f'),
        alt.Tooltip('count:Q', title='Expeditions')
    ]
).properties(
    title='Relationship Between Expedition Duration and Success',
    width=600,
    height=400
).transform_filter(
    duration_peak_selector
)

# Gráfico para tasas de éxito por bin de duración y temporada
duration_line = alt.Chart(duration_success).mark_line(point=True).encode(
    x=alt.X('duration_bin:N', axis=alt.Axis(title='Expedition Duration (days)')),
    y=alt.Y('success_rate:Q', axis=alt.Axis(title='Success Rate', format='.0%'), scale=alt.Scale(domain=[0, 1])),
    color=alt.Color('SEASON_FACTOR:N', legend=alt.Legend(title='Season')),
    strokeWidth=alt.value(3),
    tooltip=[
        alt.Tooltip('PKNAME:N', title='Peak'),
        alt.Tooltip('SEASON_FACTOR:N', title='Season'),
        alt.Tooltip('duration_bin:N', title='Duration (days)'),
        alt.Tooltip('success_rate:Q', title='Success Rate', format='.1%'),
        alt.Tooltip('total:Q', title='Total Expeditions')
    ]
).properties(
    title='Success Rate by Expedition Duration and Season',
    width=600,
    height=300
).transform_filter(
    duration_peak_selector
).transform_filter(
    alt.FieldEqualPredicate(field='SEASON_FACTOR', equal=season_selector.selection['SEASON_FACTOR']) |
    (alt.datum.SEASON_FACTOR != alt.datum.SEASON_FACTOR & (season_selector.selection['SEASON_FACTOR'] == 'all')) |
    (alt.FieldEqualPredicate(field='SEASON_FACTOR', equal=alt.datum.SEASON_FACTOR) &
     (season_selector.selection['SEASON_FACTOR'] == 'all'))
)

# Histograma para la distribución de duración de expediciones
duration_hist = alt.Chart(duration_df).mark_bar().encode(
    x=alt.X('TOTDAYS:Q', bin=alt.Bin(maxbins=30), axis=alt.Axis(title='Expedition Duration (days)')),
    y=alt.Y('count():Q', axis=alt.Axis(title='Number of Expeditions')),
    color=alt.Color('ANY_SUCCESS:N',
                   scale=alt.Scale(domain=[True, False], range=['#48c13d', '#c22d2d']),
                   legend=alt.Legend(title='Summit Success')),
    tooltip=[
        alt.Tooltip('ANY_SUCCESS:N', title='Success'),
        alt.Tooltip('count():Q', title='Expeditions')
    ]
).properties(
    title='Distribution of Expedition Durations',
    width=600,
    height=200
).transform_filter(
    duration_peak_selector
).transform_filter(
    alt.FieldEqualPredicate(field='SEASON_FACTOR', equal=season_selector.selection['SEASON_FACTOR']) |
    (alt.datum.SEASON_FACTOR != alt.datum.SEASON_FACTOR & (season_selector.selection['SEASON_FACTOR'] == 'all')) |
    (alt.FieldEqualPredicate(field='SEASON_FACTOR', equal=alt.datum.SEASON_FACTOR) &
     (season_selector.selection['SEASON_FACTOR'] == 'all'))
)

# Combinar los gráficos
duration_viz = alt.vconcat(
    scatter_chart,
    alt.vconcat(duration_line, duration_hist).resolve_scale(color='independent')
).resolve_scale(
    color='independent'
).add_selection(
    duration_peak_selector,
    season_selector
)

duration_viz

<ipython-input-61-5b55568544e6>:107: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  ).add_selection(


alt.VConcatChart(...)

### 4. Evolución de las razones de terminación

In [62]:
# Selector para picos
# Changed from selection_single to selection_point and init to value
term_peak_selector = alt.selection_point(
    name='term_peak_select',
    fields=['PEAKID'],
    value=[{'PEAKID': top_peaks[0]}], # Use 'value' instead of 'init' and pass a list of dictionaries
    bind=alt.binding_select(options=top_peaks, name='Peak: ')
)

# Selector para razones de terminación
reason_selector = alt.selection_multi(
    fields=['reason_grouped'],
    bind='legend'
)

# Gráfico de área apilada para la evolución de razones
termination_area = alt.Chart(term_evolution).mark_area().encode(
    x=alt.X('period:N', axis=alt.Axis(title='Time Period', labelAngle=-45)),
    y=alt.Y('percentage:Q', axis=alt.Axis(title='Percentage of Expeditions'), stack='normalize'),
    color=alt.Color('reason_grouped:N',
                   scale=alt.Scale(scheme='category20'),
                   legend=alt.Legend(title='Termination Reason')),
    opacity=alt.condition(reason_selector, alt.value(1), alt.value(0.2)),
    tooltip=[
        alt.Tooltip('PKNAME:N', title='Peak'),
        alt.Tooltip('period:N', title='Period'),
        alt.Tooltip('reason_grouped:N', title='Termination Reason'),
        alt.Tooltip('percentage:Q', title='Percentage', format='.1f'),
        alt.Tooltip('count:Q', title='Expeditions'),
        alt.Tooltip('total:Q', title='Total in Period')
    ]
).properties(
    title='Evolution of Termination Reasons Over Time',
    width=800,
    height=400
).transform_filter(
    term_peak_selector
).add_selection(
    reason_selector
)

# Gráfico de líneas para la evolución de razones específicas
termination_line = alt.Chart(term_evolution).mark_line(point=True).encode(
    x=alt.X('period:N', axis=alt.Axis(title='Time Period', labelAngle=-45)),
    y=alt.Y('percentage:Q', axis=alt.Axis(title='Percentage of Expeditions')),
    color=alt.Color('reason_grouped:N', scale=alt.Scale(scheme='category20'), legend=None),
    strokeWidth=alt.condition(reason_selector, alt.value(3), alt.value(1)),
    opacity=alt.condition(reason_selector, alt.value(1), alt.value(0.2)),
    tooltip=[
        alt.Tooltip('PKNAME:N', title='Peak'),
        alt.Tooltip('period:N', title='Period'),
        alt.Tooltip('reason_grouped:N', title='Termination Reason'),
        alt.Tooltip('percentage:Q', title='Percentage', format='.1f'),
        alt.Tooltip('count:Q', title='Expeditions'),
        alt.Tooltip('total:Q', title='Total in Period')
    ]
).properties(
    title='Trend of Specific Termination Reasons',
    width=800,
    height=300
).transform_filter(
    term_peak_selector
).add_selection(
    reason_selector
)

# Gráfico de barras para totales generales
termination_bars = alt.Chart(term_evolution).mark_bar().encode(
    x=alt.X('reason_grouped:N', axis=alt.Axis(title='Termination Reason', labelAngle=-45)),
    y=alt.Y('sum(count):Q', axis=alt.Axis(title='Number of Expeditions')),
    color=alt.Color('reason_grouped:N', scale=alt.Scale(scheme='category20'), legend=None),
    opacity=alt.condition(reason_selector, alt.value(1), alt.value(0.2)),
    tooltip=[
        alt.Tooltip('reason_grouped:N', title='Termination Reason'),
        alt.Tooltip('sum(count):Q', title='Expeditions')
    ]
).properties(
    title='Total Expeditions by Termination Reason',
    width=800,
    height=200
).transform_filter(
    term_peak_selector
).add_selection(
    reason_selector
)

# Combinar los gráficos
termination_viz = alt.vconcat(
    termination_area,
    termination_line,
    termination_bars
).resolve_scale(
    color='shared'
).add_selection(
    term_peak_selector
)

termination_viz

<ipython-input-62-aef5bb4e18dc>:11: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  reason_selector = alt.selection_multi(
<ipython-input-62-aef5bb4e18dc>:38: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  ).add_selection(
<ipython-input-62-aef5bb4e18dc>:63: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  ).add_selection(
<ipython-input-62-aef5bb4e18dc>:83: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  ).add_selection(
<ipython-input-62-aef5bb4e18dc>:94: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  ).add_selection(


alt.VConcatChart(...)

## Visualización Integrada Multi-Vista

Ahora, integramos las visualizaciones anteriores en un sistema coordinado que permita responder a las preguntas principales del proyecto.

In [63]:
# Crear un selector de picos compartido entre todas las vistas
# Changed from selection_single to selection_point and init to value for Altair v5
shared_peak_selector = alt.selection_point(
    name='peak_selection',
    fields=['PEAKID'],
    # Use 'value' instead of 'init' and pass a list of dictionaries
    value=[{'PEAKID': top_peaks[0]}],
    bind=alt.binding_select(options=top_peaks, name='Mountain Peak: ')
)

# Crear un mapa simple con los picos del Himalaya
base_map = alt.Chart(coords_df).mark_circle(size=100).encode(
    longitude='LONGITUDE:Q',
    latitude='LATITUDE:Q',
    color=alt.condition(
        shared_peak_selector,
        alt.value('red'),
        alt.value('steelblue')
    ),
    tooltip=[
        alt.Tooltip('PKNAME:N', title='Peak Name'),
        alt.Tooltip('PEAKID:N', title='Peak ID')
    ]
).properties(
    width=300,
    height=300,
    title='Himalayan Peaks Location'
).project(
    type='mercator'
).add_selection(
    shared_peak_selector
)

# Adaptar los gráficos anteriores para usar el selector compartido
# 1. Gráfico de rutas y tasas de éxito
route_chart_shared = alt.Chart(route_success_rates).mark_bar().encode(
    x=alt.X('ROUTE:N', axis=alt.Axis(title='Route', labelAngle=-45), sort='-y'),
    y=alt.Y('success_rate:Q', axis=alt.Axis(title='Success Rate', format='.0%'), scale=alt.Scale(domain=[0, 1])),
    color=alt.Color('success_rate:Q',
                   scale=alt.Scale(domain=[0, 0.25, 0.5, 0.75, 1.0],
                                  range=['#c22d2d', '#e77e16', '#ffb533', '#d9e03f', '#48c13d']),
                   legend=alt.Legend(title='Success Rate')),
    tooltip=[
        alt.Tooltip('PKNAME:N', title='Peak'),
        alt.Tooltip('ROUTE:N', title='Route'),
        alt.Tooltip('success_rate:Q', title='Success Rate', format='.1%'),
        alt.Tooltip('successful_attempts:Q', title='Successful'),
        alt.Tooltip('total_attempts:Q', title='Total Attempts')
    ]
).properties(
    title='Success Rates by Route',
    width=600,
    height=300
).transform_filter(
    shared_peak_selector
)

# 2. Gráfico de duración y éxito
duration_line_shared = alt.Chart(duration_success).mark_line(point=True).encode(
    x=alt.X('duration_bin:N', axis=alt.Axis(title='Expedition Duration (days)')),
    y=alt.Y('success_rate:Q', axis=alt.Axis(title='Success Rate', format='.0%'), scale=alt.Scale(domain=[0, 1])),
    color=alt.Color('SEASON_FACTOR:N', legend=alt.Legend(title='Season')),
    strokeWidth=alt.value(3),
    tooltip=[
        alt.Tooltip('PKNAME:N', title='Peak'),
        alt.Tooltip('SEASON_FACTOR:N', title='Season'),
        alt.Tooltip('duration_bin:N', title='Duration (days)'),
        alt.Tooltip('success_rate:Q', title='Success Rate', format='.1%'),
        alt.Tooltip('total:Q', title='Total Expeditions')
    ]
).properties(
    title='Success Rate by Duration and Season',
    width=600,
    height=300
).transform_filter(
    shared_peak_selector
)

# 3. Gráfico de evolución de razones de terminación
term_area_shared = alt.Chart(term_evolution).mark_area().encode(
    x=alt.X('period:N', axis=alt.Axis(title='Time Period', labelAngle=-45)),
    y=alt.Y('percentage:Q', axis=alt.Axis(title='Percentage', format='.0%'), stack='normalize'),
    color=alt.Color('reason_grouped:N', scale=alt.Scale(scheme='category20'), legend=alt.Legend(title='Termination Reason')),
    tooltip=[
        alt.Tooltip('PKNAME:N', title='Peak'),
        alt.Tooltip('period:N', title='Period'),
        alt.Tooltip('reason_grouped:N', title='Reason'),
        alt.Tooltip('percentage:Q', title='Percentage', format='.1f'),
        alt.Tooltip('count:Q', title='Expeditions'),
        alt.Tooltip('total:Q', title='Total in Period')
    ]
).properties(
    title='Evolution of Termination Reasons',
    width=600,
    height=300
).transform_filter(
    shared_peak_selector
)

# 4. Información del pico seleccionado
peak_info_shared = alt.Chart(df_merged.drop_duplicates('PEAKID')).mark_bar().encode(
    y=alt.Y('PKNAME:N', axis=alt.Axis(title='Peak Name')),
    x=alt.X('HEIGHTM:Q', axis=alt.Axis(title='Height (m)')),
    color=alt.condition(shared_peak_selector, alt.value('red'), alt.value('lightgray')),
    tooltip=[
        alt.Tooltip('PKNAME:N', title='Peak Name'),
        alt.Tooltip('HEIGHTM:Q', title='Height (m)'),
        alt.Tooltip('REGION_FACTOR:N', title='Region'),
        alt.Tooltip('HIMAL_FACTOR:N', title='Mountain Range')
    ]
).properties(
    title='Peak Information',
    width=300,
    height=300
).transform_filter(
    alt.FieldOneOfPredicate(field='PEAKID', oneOf=top_peaks)
)

# 5. Expediciones por país para el pico seleccionado
country_exped_by_peak = df_merged.groupby(['PEAKID', 'PKNAME', 'HOST_FACTOR']).size().reset_index(name='count')
country_exped_by_peak = country_exped_by_peak.sort_values('count', ascending=False)

countries_chart_shared = alt.Chart(country_exped_by_peak).mark_bar().encode(
    y=alt.Y('HOST_FACTOR:N', axis=alt.Axis(title='Country'), sort='-x'),
    x=alt.X('count:Q', axis=alt.Axis(title='Number of Expeditions')),
    color=alt.Color('HOST_FACTOR:N', legend=None),
    tooltip=[
        alt.Tooltip('HOST_FACTOR:N', title='Country'),
        alt.Tooltip('PKNAME:N', title='Peak'),
        alt.Tooltip('count:Q', title='Expeditions')
    ]
).properties(
    title='Expeditions by Country',
    width=300,
    height=300
).transform_filter(
    shared_peak_selector
).transform_window(
    rank='rank(count)',
    sort=[alt.SortField('count', order='descending')]
).transform_filter(
    alt.datum.rank <= 10
)

# Organizar todos los gráficos en un dashboard integrado
row1 = alt.hconcat(base_map, peak_info_shared, countries_chart_shared)
row2 = alt.hconcat(route_chart_shared, duration_line_shared)
row3 = term_area_shared

# Dashboard final
dashboard = alt.vconcat(
    row1,
    row2,
    row3
).resolve_scale(
    color='independent'
).configure_view(
    stroke=None
)

dashboard

<ipython-input-63-52912c5c840a>:30: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  ).add_selection(


alt.VConcatChart(...)

## Exportar los datos procesados para la aplicación Streamlit

In [65]:
# Guardar los datasets procesados para la aplicación Streamlit
df_merged.to_csv(output_data+'himalayan_data_processed.csv', index=False)
route_success_rates.to_csv(output_data+'route_success_rates.csv', index=False)
duration_success.to_csv(output_data+'duration_success.csv', index=False)
term_evolution.to_csv(output_data+'termination_evolution.csv', index=False)

## Documentación del proceso de diseño

### Proceso de limpieza de datos (300 palabras)

El proceso de limpieza y preprocesamiento de datos se centró en preparar los tres conjuntos de datos (expediciones, picos y coordenadas) para el análisis visual. Los pasos principales incluyeron:

1. **Unificación de valores nulos**: Se estandarizaron las representaciones de valores nulos en las columnas de rutas ('NA', cadenas vacías y NaN) para facilitar los análisis posteriores.

2. **Conversión de tipos de datos**: Las columnas de éxito se convirtieron explícitamente a booleanos, mientras que valores numéricos como TOTDAYS y YEAR se convirtieron a formatos adecuados para cálculos.

3. **Creación de variables derivadas**: Se generó un indicador de éxito general (ANY_SUCCESS) para determinar si una expedición tuvo éxito en cualquiera de sus rutas. También se crearon bins para análisis temporales (décadas, períodos de 5 años) y de duración.

4. **Filtrado de datos incompletos**: Se eliminaron registros que carecían de información esencial como el identificador del pico o el año de la expedición.

5. **Combinación de conjuntos de datos**: Se unieron los tres datasets para tener una visión completa de cada expedición, incluyendo características geográficas y físicas de los picos.

6. **Normalización para análisis específicos**: Para cada pregunta de investigación, se prepararon conjuntos de datos especializados:
   - Para el análisis de rutas, se expandieron los datos para tener una fila por cada combinación ruta-expedición.
   - Para el análisis de duración, se establecieron categorías significativas de duración.
   - Para el análisis de razones de terminación, se agruparon las razones menos comunes en una categoría "Otras".

7. **Filtrado para visualizaciones significativas**: Se implementaron umbrales mínimos (como 5 intentos por ruta o 3 expediciones por bin de duración) para asegurar que las estadísticas visualizadas fueran significativas.

Este proceso resultó en conjuntos de datos limpios, estructurados y listos para ser utilizados en visualizaciones interactivas, manteniendo la integridad de la información original mientras se abordaban problemas comunes como valores faltantes y formatos inconsistentes.

### Diseño de visualización para la Pregunta 1: Tasas de éxito por ruta (300 palabras)

**Chart selection y razonamiento**: Para visualizar las tasas de éxito por ruta, elegí un gráfico de barras horizontales donde cada barra representa una ruta específica para un pico seleccionado. Este tipo de gráfico es ideal porque permite comparar fácilmente valores numéricos (tasas de éxito) entre categorías (rutas), y las barras ordenadas facilitan identificar rápidamente las rutas más y menos exitosas.

**Interactividad**: Implementé un selector desplegable para elegir el pico a analizar, lo que permite al usuario explorar diferentes montañas sin saturar la visualización. El color codifica la tasa de éxito usando una escala que va del rojo (bajo éxito) al verde (alto éxito), proporcionando una capa adicional de información visual. Los tooltips detallados muestran información contextual como el número total de intentos y éxitos para cada ruta.

**Iteraciones de diseño**: Inicialmente, intenté mostrar todas las rutas para todos los picos simultáneamente, pero esto resultaba abrumador y poco informativo. Posteriormente, implementé el filtrado interactivo por pico. También experimenté con diferentes esquemas de color antes de decidirme por el espectro rojo-verde, que comunica intuitivamente el concepto de éxito/fracaso.

**Mejoras de legibilidad**: Para mejorar la legibilidad, ordené las rutas por tasa de éxito descendente, lo que permite identificar inmediatamente las rutas más exitosas. Agregué etiquetas claras en los ejes y un título descriptivo. Para manejar nombres de rutas largos, implementé etiquetas en ángulo de 45 grados.

**Diseños alternativos**: Consideré usar gráficos de puntos o heatmaps para visualizar esta información, pero el gráfico de barras resultó más efectivo para comunicar comparaciones directas. También evalué usar colores categóricos para cada ruta, pero el esquema de color basado en valores resultó más informativo al transmitir tanto la identidad de la ruta como su tasa de éxito.

Esta visualización permite responder eficientemente a la pregunta sobre qué rutas tienen las tasas de éxito más altas y más bajas para diferentes picos, proporcionando un mecanismo interactivo para explorar patrones a través de diferentes montañas del Himalaya.

### Diseño de visualización para la Pregunta 2: Expediciones por país (300 palabras)

**Chart selection y razonamiento**: Para analizar qué países han liderado más expediciones a lo largo del tiempo, implementé un gráfico de líneas con puntos donde cada línea representa un país. Este tipo de gráfico es ideal para visualizar tendencias temporales y permite identificar patrones, puntos de inflexión y comparaciones entre países. Como complemento, añadí un gráfico de barras apiladas para mostrar la composición total de expediciones por década.

**Interactividad**: La interactividad principal es la selección por leyenda, que permite resaltar países específicos haciendo clic en la leyenda. Esto facilita el seguimiento de la evolución de países individuales sin eliminar el contexto general. Los tooltips proporcionan información precisa sobre el número de expediciones para cada país en cada década.

**Iteraciones de diseño**: Inicialmente mostré todos los países, pero esto generaba un gráfico saturado con más de 50 líneas. Refine el diseño para mostrar solo los 10 países con más expediciones, lo que mejora significativamente la legibilidad. También experimenté con diferentes agrupaciones temporales antes de decidirme por décadas, que ofrecen un buen equilibrio entre detalle y visión general.

**Mejoras de legibilidad**: Para mejorar la claridad visual, implementé un cambio de grosor y opacidad en las líneas cuando se seleccionan países, lo que permite destacar las naciones de interés. También utilicé colores distintivos para cada país y añadí puntos en los datos para facilitar la lectura de valores específicos.

**Diseños alternativos**: Consideré usar un mapa coroplético mundial, pero un gráfico de líneas resultó más efectivo para visualizar cambios temporales. También evalué usar gráficos de áreas apiladas, pero el solapamiento dificultaba la interpretación de tendencias para países específicos.

Esta visualización permite observar eficazmente cómo ha cambiado la participación de diferentes países en expediciones himaláyicas a lo largo del tiempo, revelando patrones históricos como el dominio inicial de países occidentales y el posterior aumento de expediciones lideradas por países asiáticos como Nepal, China e India.

### Diseño de visualización para la Pregunta 3: Duración y éxito (300 palabras)

**Chart selection y razonamiento**: Para analizar la relación entre duración de expediciones y tasas de éxito, implementé un conjunto de visualizaciones complementarias: un gráfico de líneas que muestra las tasas de éxito por categoría de duración y temporada, un gráfico de dispersión que compara la duración promedio de expediciones exitosas vs no exitosas, y un histograma que muestra la distribución general de duraciones.

**Interactividad**: La visualización incluye selectores para elegir el pico y la temporada de interés. El selector de temporada incluye una opción "all" para ver todas las temporadas simultáneamente, facilitando comparaciones. Esta interactividad permite explorar cómo la relación entre duración y éxito varía entre diferentes montañas y condiciones estacionales.

**Iteraciones de diseño**: Inicialmente solo usé un gráfico de líneas, pero añadí el gráfico de dispersión y el histograma para proporcionar contexto adicional sobre la distribución de duraciones y éxitos. También experimenté con diferentes categorizaciones de duración antes de establecer bins significativos que balancean detalle y tamaño de muestra adecuado.

**Mejoras de legibilidad**: Para mejorar la interpretación, implementé un esquema de colores consistente donde verde indica éxito y rojo fracaso. Los tooltips detallados muestran información contextual como el número de expediciones en cada categoría. También añadí títulos descriptivos y etiquetas claras en los ejes.

**Diseños alternativos**: Consideré usar gráficos de caja para mostrar la distribución de duraciones por resultado, pero los gráficos seleccionados resultaron más efectivos para comunicar tanto los patrones generales como las tendencias específicas por temporada. También evalué usar un gráfico de calor para mostrar la relación entre duración, temporada y éxito, pero resultaba menos intuitivo.

Esta visualización integrada permite responder efectivamente a la pregunta sobre si las expediciones más largas tienen mayor probabilidad de éxito, y cómo esta relación varía según la temporada y el pico específico, revelando patrones como la existencia de una "duración óptima" que varía según las condiciones estacionales.

### Diseño de visualización para la Pregunta 4: Evolución de razones de terminación (300 palabras)

**Chart selection y razonamiento**: Para visualizar la evolución de las razones de terminación a lo largo del tiempo, implementé un gráfico de áreas apiladas normalizado como visualización principal. Este tipo de gráfico es ideal para mostrar cómo ha cambiado la composición porcentual de diferentes categorías a lo largo del tiempo. Complementé esto con un gráfico de líneas que permite seguir tendencias específicas y un gráfico de barras que muestra los totales generales.

**Interactividad**: La interactividad principal es la selección por leyenda, que permite destacar razones de terminación específicas. También incluí un selector para elegir el pico a analizar. Esta combinación facilita tanto la exploración general como el análisis detallado de razones específicas para montañas concretas.

**Iteraciones de diseño**: Inicialmente mostré todas las razones de terminación individualmente, pero esto creaba un gráfico saturado con demasiadas categorías. Refiné el diseño agrupando las razones menos comunes en una categoría "Otras razones", lo que mejoró significativamente la legibilidad. También experimenté con diferentes agrupaciones temporales antes de decidirme por períodos de 5 años, que proporcionan suficiente detalle temporal sin fragmentar excesivamente los datos.

**Mejoras de legibilidad**: Para mejorar la interpretación, implementé cambios de opacidad y grosor de línea cuando se seleccionan razones específicas. Añadí tooltips detallados que muestran tanto valores absolutos como porcentajes para cada categoría en cada período. Utilicé colores distintivos para cada razón y añadí títulos claros.

**Diseños alternativos**: Consideré usar solo gráficos de líneas o solo gráficos de barras apiladas, pero la combinación de técnicas resultó más efectiva para comunicar tanto la composición porcentual (áreas) como las tendencias absolutas (líneas) y los totales (barras). También evalué usar animaciones temporales, pero el formato estático con controles interactivos resultó más práctico.

Esta visualización permite analizar eficazmente cómo han evolucionado las razones por las que terminan las expediciones, revelando patrones como el aumento de expediciones exitosas en años recientes o cambios en la prevalencia de ciertos desafíos como mal tiempo o enfermedades para diferentes picos.

### Diseño del sistema multi-vista integrado (500 palabras)

**Enfoque de diseño**: El diseño del sistema multi-vista integrado se basó en el principio de proporcionar múltiples perspectivas coordinadas sobre los datos de expediciones himaláyicas, permitiendo a los usuarios explorar las cuatro preguntas clave del PRD de manera coherente y complementaria. La arquitectura general siguió un enfoque de "overview + detail", donde la fila superior proporciona contexto general sobre los picos y expediciones, mientras que las filas inferiores ofrecen visualizaciones detalladas para responder a preguntas específicas.

**Componentes principales y organización espacial**:

1. **Fila superior (Contexto general)**:
   - Mapa geográfico que muestra la ubicación de los picos con interactividad para seleccionar montañas.
   - Gráfico de barras con información de altura y región de los picos seleccionables.
   - Distribución de expediciones por país para el pico seleccionado.
   
   Esta fila proporciona contexto espacial, físico y geopolítico, estableciendo la base para el análisis detallado.

2. **Fila intermedia (Rutas y Duración)**:
   - Gráfico de barras para tasas de éxito por ruta, respondiendo a la Pregunta 1.
   - Gráfico de líneas para la relación entre duración y éxito por temporada, abordando la Pregunta 3.
   
   Esta fila permite comparar factores operativos clave que influyen en el éxito de las expediciones.

3. **Fila inferior (Evolución temporal)**:
   - Gráfico de áreas para la evolución de razones de terminación, respondiendo a la Pregunta 4.
   
   Esta fila proporciona la perspectiva histórica y evolutiva de los desafíos enfrentados por los expedicionarios.

**Coordinación entre vistas**: El elemento clave que une todas las visualizaciones es un selector compartido de picos que actualiza simultáneamente todas las vistas cuando se selecciona una montaña diferente. Esta coordinación permite a los usuarios mantener el contexto mientras exploran diferentes aspectos de un mismo pico, facilitando la identificación de patrones y relaciones entre diferentes variables.

**Consideraciones de color y consistencia visual**: Se mantuvieron esquemas de color consistentes a través de las visualizaciones (verde para éxito, rojo para fracaso/selección activa), lo que facilita la interpretación de los datos y crea cohesión visual. Cada visualización mantiene su propia escala de color para variables específicas, pero con una estética general unificada.

**Iteraciones y mejoras**:

1. **Versión inicial**: La primera implementación tenía selectores independientes para cada visualización, lo que hacía difícil mantener la coherencia entre vistas.

2. **Mejora de coordinación**: Se implementó un selector compartido que permitía actualizar todas las vistas simultáneamente.

3. **Optimización espacial**: Se ajustaron los tamaños de las visualizaciones para maximizar el uso de espacio sin crear desplazamiento excesivo, siguiendo la restricción del PRD de limitar el desplazamiento vertical.

4. **Mejora de contexto inicial**: Se añadió el mapa y la información de picos en la fila superior para proporcionar contexto geográfico y físico inmediato.

**Instrucciones paso a paso para resolver cada tarea**:

1. **Para identificar rutas con mayor/menor tasa de éxito**:
   - Seleccionar el pico de interés en el desplegable superior.
   - Observar el gráfico de barras de rutas en la fila intermedia izquierda, donde las rutas están ordenadas por tasa de éxito.
   - Pasar el cursor sobre las barras para ver información detallada sobre cada ruta.

2. **Para analizar qué países han liderado más expediciones**:
   - Seleccionar el pico de interés en el desplegable superior.
   - Revisar el gráfico de barras de países en la fila superior derecha, donde se muestran los 10 países principales ordenados por número de expediciones.

3. **Para determinar si expediciones más largas tienen mayor probabilidad de éxito**:
   - Seleccionar el pico de interés en el desplegable superior.
   - Examinar el gráfico de líneas en la fila intermedia derecha, donde se muestra la tasa de éxito por categoría de duración y temporada.
   - Identificar si hay una tendencia clara (ascendente, descendente o con pico) en la relación entre duración y éxito.

4. **Para analizar la evolución de razones de terminación**:
   - Seleccionar el pico de interés en el desplegable superior.
   - Observar el gráfico de áreas en la fila inferior, que muestra cómo ha cambiado la distribución de razones a lo largo del tiempo.
   - Hacer clic en razones específicas en la leyenda para resaltarlas y analizar sus tendencias con mayor detalle.

Este sistema multi-vista integrado permite responder eficientemente a todas las preguntas clave del proyecto mientras proporciona contexto adicional que enriquece el análisis, todo ello respetando las restricciones de usabilidad y navegación especificadas en el PRD.